<a href="https://colab.research.google.com/github/faranak5171/Deep-Learning-projects/blob/master/Dogs_vs_Cats_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torchvision import transforms, datasets
from torch import nn, optim
import torch.nn.functional as F

In [2]:
! pip install kaggle
! mkdir ~/.kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
! cp kaggle.json ~/.kaggle/
! cp kaggle.json ~/.kaggle/

In [4]:
! kaggle competitions download dogs-vs-cats

 97% 788M/812M [00:03<00:00, 203MB/s]
100% 812M/812M [00:03<00:00, 269MB/s]


In [5]:
! unzip dogs-vs-cats

Archive:  dogs-vs-cats.zip
  inflating: sampleSubmission.csv    
  inflating: test1.zip               
  inflating: train.zip               
